In [5]:
import sys
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.applications import Knapsack

In [5]:
def optimize_battery_revenue(values, weights, c_max):
    """
    Optimize battery revenue with a strict C_max constraint.

    Args:
        values (list[list[float]]): 2D list where values[i][j] is the revenue of market i at time j.
        weights (list[list[float]]): 2D list where weights[i][j] is the weight of market i at time j.
        c_max (float): Maximum allowable weight.

    Returns:
        tuple: Optimal solution (list of market choices) and the maximum revenue.
    """
    n = len(values[0])  # Number of time periods
    num_markets = len(values)  # Number of markets
    
    def backtrack(index, current_weight, current_revenue, current_solution):
        if index == n:
            return current_solution, current_revenue

        best_solution, best_revenue = None, -float('inf')

        for market in range(num_markets):
            new_weight = current_weight + weights[market][index]
            new_revenue = current_revenue + values[market][index]

            if new_weight <= c_max:
                solution, revenue = backtrack(
                    index + 1, 
                    new_weight, 
                    new_revenue, 
                    current_solution + [market]
                )

                if revenue > best_revenue:
                    best_solution, best_revenue = solution, revenue

        return best_solution, best_revenue

    solution, revenue = backtrack(0, 0, 0, [])
    return solution, revenue

def formulate_qp(values, weights, c_max):
    """
    Formulate the knapsack problem as a quadratic program.

    Args:
        values (list[list[float]]): Revenue values for markets.
        weights (list[list[float]]): Weight values for markets.
        c_max (float): Maximum allowable weight.

    Returns:
        QuadraticProgram: The formulated quadratic program.
    """
    from qiskit_optimization import QuadraticProgram

    n = len(values[0])  # Number of time periods
    num_markets = len(values)  # Number of markets
    qp = QuadraticProgram("Knapsack Problem")

    # Define binary variables
    for i in range(n):
        for market in range(num_markets):
            qp.binary_var(name=f"M{market}_{i}")

    # Objective function: Maximize profits
    linear_terms = {
        f"M{market}_{i}": values[market][i]
        for market in range(num_markets)
        for i in range(n)
    }
    qp.maximize(linear=linear_terms)

    # Constraint: Each item must be in exactly one market
    for i in range(n):
        qp.linear_constraint(
            linear={f"M{market}_{i}": 1 for market in range(num_markets)},
            sense="==",
            rhs=1,
            name=f"state_constraint_{i}",
        )

    # Constraint: Total weight must not exceed C_max
    weight_terms = {
        f"M{market}_{i}": weights[market][i]
        for market in range(num_markets)
        for i in range(n)
    }
    qp.linear_constraint(
        linear=weight_terms,
        sense="<=",
        rhs=c_max,
        name="weight_constraint",
    )

    return qp

def solve_qp_with_qiskit(values, weights, c_max):
    """
    Solve the quadratic program using Qiskit's optimization solvers.

    Args:
        values (list[list[float]]): Revenue values for markets.
        weights (list[list[float]]): Weight values for markets.
        c_max (float): Maximum allowable weight.

    Returns:
        dict: Solution details including selected items and total revenue.
    """
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_optimization.algorithms import CobylaOptimizer
    from qiskit_optimization.algorithms import QAOA

    qp = formulate_qp(values, weights, c_max)

    print(qp.prettyprint())  # Optional: Print the formulated QP

    optimizer = MinimumEigenOptimizer(CobylaOptimizer())

    result = optimizer.solve(qp)
    solution = result.x

    total_profit = 0
    assignments = []

    num_markets = len(values)
    n = len(values[0])

    for i in range(n):
        for market in range(num_markets):
            if solution[num_markets * i + market] == 1:
                total_profit += values[market][i]
                assignments.append(f"M{market}_{i}")

    return {
        "solution": solution,
        "assignments": assignments,
        "total_profit": total_profit,
    }


In [34]:
def knapsack_argument(total_value_list, total_weight_list, weight_max):
    max_combination_list_idx = []
    
    length = len(total_value_list[0])
    updated_combination_list_idx = [0] * length
    updated_combination_values = total_value_list[0].copy()
    updated_combination_weights = total_weight_list[0].copy()
    
    for i in range(len(total_value_list)-1):
        values = list(total_value_list[i+1][j]- updated_combination_values[j] for j in range(length))
        weights = list(total_weight_list[i+1][j]- updated_combination_weights[j] for j in range(length))
        max_weight = weight_max - sum(updated_combination_weights) # 1 선택을 기본 전제로 깔고, 위 values & weights는 2를 선택했을 때의 상대적 이득을 의미하게 됨
    
        prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
        kqp = prob.to_quadratic_program()
        
        mes = QAOA(sampler=Sampler(), optimizer=COBYLA())
        meo = MinimumEigenOptimizer(min_eigen_solver=mes)
        
        result = meo.solve(kqp)
                
        for j in range(len(result.x)):
            if result.x[j] == 1:
                updated_combination_list_idx[j] = i+1
                updated_combination_values[j] = total_value_list[i+1][j]
                updated_combination_weights[j] = total_weight_list[i+1][j]
    
    return updated_combination_list_idx, updated_combination_values, updated_combination_weights

In [ ]:
while True:
    num_of_options = input("How many comparative groups do you want to create?: (press enter if you want to quit) ")
    if num_of_options == "":
        try:
            sys.exit()  # 프로그램 종료
            print(1)  # 실행되지 않음
        except SystemExit:
            pass  # SystemExit 예외를 무시
    if num_of_options.isdigit():  # 숫자인지 확인
        num_of_options = int(num_of_options)
        break
    else:
        print("Please enter a valid number.")

while True:
    num_of_elements = input("How many elements are in each group?: (press enter if you want to quit) ")
    if num_of_elements == "":
        try:
            sys.exit()  # 프로그램 종료
            print(1)  # 실행되지 않음
        except SystemExit:
            pass  # SystemExit 예외를 무시
    if num_of_elements.isdigit():  # 숫자인지 확인
        num_of_elements = int(num_of_elements)
        break
    else:
        print("Please enter a valid number.")
        
while True:
    weight_max = input("What is the maximum allowable weights?: (press enter if you want to quit) ")
    if weight_max == "":
        try:
            sys.exit()  # 프로그램 종료
            print(1)  # 실행되지 않음
        except SystemExit:
            pass  # SystemExit 예외를 무시
    try:
        weight_max = float(weight_max)
        break
    except ValueError:
        print("Please enter a valid number.")


total_value_list = []
total_weight_list = []

for i in range(num_of_options):
    while True:
        values = input(f"Input values for group {i} out of {num_of_options} gropus (separate by spaces, {num_of_elements} values): ")
        value_list = values.split()  # 입력을 공백으로 나눔
        if len(value_list) != num_of_elements:  # 입력 길이 확인
            print(f"Please input exactly {num_of_elements} values.")
            continue
        try:
            value_list = [float(value) for value in value_list]  # 실수 변환
            break
        except ValueError:
            print("All values must be numbers. Please try again.")
            
    while True:
        weights = input(f"Input weights for group {i} out of {num_of_options} gropus (separate by spaces, {num_of_elements} values): ")
        weight_list = weights.split()  # 입력을 공백으로 나눔
        if len(weight_list) != num_of_elements:  # 입력 길이 확인
            print(f"Please input exactly {num_of_elements} weights.")
            continue
        try:
            weight_list = [float(weight) for weight in weight_list]  # 실수 변환
            break
        except ValueError:
            print("All weights must be numbers. Please try again.")

    total_value_list.append(value_list)
    total_weight_list.append(weight_list)

# n번 인덱스는 n번째 비교균을 의미
print("Total value list:", total_value_list)
print("Total weight list:", total_weight_list)
    
# kanpsack argument   
list_combination_idx, combination_values, combination_weights = knapsack_argument(total_value_list,total_weight_list, weight_max)
print(f"values : {combination_values}, weights: {combination_weights}")
print(list_combination_idx)
print("Total value:", sum(combination_values))

TypeError: 'str' object cannot be interpreted as an integer